# Functional Test 3.1.2 - Shared NICs

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.1.2 for testing Shared NIC attachment.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Step 3: Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [ ]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the Test Slice

The following creates a node with shared NIC. This should be run all worker nodes regardless of type.

One node with one NIC component is created.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. 


In [ ]:
from datetime import datetime
from dateutil import tz

name='Node1'
nic1_name='SharedNIC1'
site='TACC'
# since all workers have a standard naming scheme, you can just change the worker
# to move from worker to worker
worker=f'{site.lower()}-w1.fabric-testbed.net'
cores=10
ram=20
disk=50
slice_name=f"Slice Test 3.1.2-SharedNIC {site} on {worker} on {datetime.now()}"

In [ ]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=name, site=site)
    iface1 = node1.add_component(model='NIC_Basic', name=nic1_name).get_interfaces()[0]
 
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Step 5: Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node List

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node Details

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Interfaces

You should see 1 interface.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

### Shared NIC PCI Devices

Run the command <code>lspci</code> to see your Shared NIC PCI device(s).

View node1's Shared NIC - you should see a `Ethernet controller: Mellanox Technologies MT28908 Family [ConnectX-6 Virtual Function]` 

In [ ]:
command = "sudo dnf install -q -y pciutils && lspci"
try:
    stdout, stderr = node.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")